# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
from ipywidgets.embed import embed_minimal_html
# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_weather_df = pd.read_csv("../WeatherPy/Output/cities.csv")
cities_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Datetime
0,Omboué,-1.57,9.26,76.35,91,88,5.70,GA,1607313768
1,Zacualpan,21.25,-105.17,72.00,88,90,3.83,MX,1607313768
2,Fortuna,40.60,-124.16,46.00,93,38,3.74,US,1607313769
3,Norman Wells,65.28,-126.83,1.40,91,75,2.24,CA,1607313543
4,Punta Arenas,-53.15,-70.92,59.00,67,100,17.22,CL,1607313433
...,...,...,...,...,...,...,...,...,...
579,Boddam,57.47,-1.78,34.00,100,75,1.12,GB,1607313864
580,Kalyazin,57.24,37.84,17.35,95,1,7.27,RU,1607313864
581,Shwebo,22.57,95.70,76.86,60,0,3.18,MM,1607313864
582,Kloulklubed,7.04,134.26,86.00,74,75,6.93,PW,1607313864


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_weather_df[["Lat", "Lng"]]

# Store Humidity in humidity series
humidity = cities_weather_df["Humidity"].astype(float)

# Calculating the max humidity for heatmap
maxhumidity = humidity.max()

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=maxhumidity,
                                    point_radius=3.0, dissipating = False)

# Add layer
fig.add_layer(heatmap_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
sliced_city_df = cities_weather_df.loc[(cities_weather_df["Wind Speed"] <= 10) & (cities_weather_df["Cloudiness"] == 0) &
                                   (cities_weather_df["Max Temp"] >= 70) & (cities_weather_df["Max Temp"] <= 80) & 
                                      (cities_weather_df["Humidity"]<=70)].reset_index(drop= True)

sliced_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Datetime
0,Busselton,-33.65,115.33,79.00,30,0,2.75,AU,1607313599
1,Arlit,18.74,7.39,71.20,24,0,4.52,NE,1607313777
2,Hong Kong,22.29,114.16,71.01,62,0,1.01,HK,1607313612
3,Yenangyaung,20.47,94.88,77.49,51,0,3.47,MM,1607313808
4,Tonk,26.17,75.78,74.43,24,0,3.85,IN,1607313813
5,Podor,16.65,-14.96,71.60,46,0,2.24,MR,1607313817
6,Ormara,25.21,64.64,76.82,67,0,4.21,PK,1607313840
7,Malakal,9.53,31.66,73.69,21,0,7.16,SS,1607313854
8,Pālkonda,18.60,83.75,78.06,50,0,3.24,IN,1607313857
9,Shwebo,22.57,95.70,76.86,60,0,3.18,MM,1607313864


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = sliced_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Busselton,AU,-33.65,115.33,
1,Arlit,NE,18.74,7.39,
2,Hong Kong,HK,22.29,114.16,
3,Yenangyaung,MM,20.47,94.88,
4,Tonk,IN,26.17,75.78,
5,Podor,MR,16.65,-14.96,
6,Ormara,PK,25.21,64.64,
7,Malakal,SS,9.53,31.66,
8,Pālkonda,IN,18.60,83.75,
9,Shwebo,MM,22.57,95.70,


In [7]:
# URL for places API
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(f"{url}?location={lat},{lng}&radius=5000&keyword=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 0: Busselton.
Closest hotel in Busselton is Abbey Beach Resort.
------------
Retrieving Results for Index 1: Arlit.
Closest hotel in Arlit is Hôtel Telwa Bungalow.
------------
Retrieving Results for Index 2: Hong Kong.
Closest hotel in Hong Kong is Very Good Hotel.
------------
Retrieving Results for Index 3: Yenangyaung.
Closest hotel in Yenangyaung is Lei Thar Gone Guest House.
------------
Retrieving Results for Index 4: Tonk.
Closest hotel in Tonk is The Om Vilas Hotel.
------------
Retrieving Results for Index 5: Podor.
Closest hotel in Podor is La cour du fleuve.
------------
Retrieving Results for Index 6: Ormara.
Closest hotel in Ormara is Alqadri hotel zero point ormara.
------------
Retrieving Results for Index 7: Malakal.
Closest hotel in Malakal is Nile Palace Hotel.
------------
Retrieving Results for Index 8: Pālkonda.
Closest hotel in Pālkonda is Lodge Uma Sankar.
------------
Retrieving Results for Index 9: Shwebo.
Closest hotel in Shwebo i

In [9]:
# Excluding the cities where the API has not returned any hotel
hotel_df = hotel_df[hotel_df["Hotel Name"] != ""].reset_index(drop= True)
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Busselton,AU,-33.65,115.33,Abbey Beach Resort
1,Arlit,NE,18.74,7.39,Hôtel Telwa Bungalow
2,Hong Kong,HK,22.29,114.16,Very Good Hotel
3,Yenangyaung,MM,20.47,94.88,Lei Thar Gone Guest House
4,Tonk,IN,26.17,75.78,The Om Vilas Hotel
5,Podor,MR,16.65,-14.96,La cour du fleuve
6,Ormara,PK,25.21,64.64,Alqadri hotel zero point ormara
7,Malakal,SS,9.53,31.66,Nile Palace Hotel
8,Pālkonda,IN,18.60,83.75,Lodge Uma Sankar
9,Shwebo,MM,22.57,95.70,Shwe Pyi ThuKha Guest House


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info)

# Add the markers as a layer
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Export the map in an interactive HTML file
embed_minimal_html("Images/hotels.html", views=[fig])